# Hourly Data #

In [1]:
# imports
import pandas as pd
import numpy as np
import time

In [2]:
# define data location
location = '/Users/mithras/Documents/_SCHOOL/_Drexel/BUSN 710 - Capstone/Data/Forecasting Project/'
#location = '/Users/loki/Documents/Data/Forecasting Project/'

## Data Cleaning & Checking 

In [3]:
# hourly data
use_jan_in = pd.read_excel(location+'Zip_HourlylUsage_2018.01.xlsx')
use_jul_in = pd.read_excel(location+'Zip_HourlylUsage_2018.07.xlsx')

# daily data
use_oct_in = pd.read_excel(location+'Sample Usage_2017.10 Oct.xlsx')
use_nov_in = pd.read_excel(location+'Sample Usage_2017.11 Nov.xlsx')
use_dec_in = pd.read_excel(location+'Sample Usage_2017.12 Dec.xlsx')
# use_jan_in = pd.read_excel(location+'Sample Usage_2018.01 Jan.xlsx')
use_feb_in = pd.read_excel(location+'Sample Usage_2018.02 Feb.xlsx')
use_mar_in = pd.read_excel(location+'Sample Usage_2018.03 March.xlsx')
use_apr_in = pd.read_excel(location+'Sample Usage_2018.04 April.xlsx')
use_may_in = pd.read_excel(location+'Sample Usage_2018.05 May.xlsx')
use_jun_in = pd.read_excel(location+'Sample Usage_2018.06 June.xlsx')
# use_jul_in = pd.read_excel(location+'Sample Usage_2018.07 Jul.xlsx')
use_aug_in = pd.read_excel(location+'Sample Usage_2018.08 Aug.xlsx')
use_sep_in = pd.read_excel(location+'Sample Usage_2018.09 Sep.xlsx')

# weather data
temp_in = pd.read_excel(location+'Weather Data for Drexel 9_28_2018.xlsx', sheet_name="TMP")
humid_in = pd.read_excel(location+'Weather Data for Drexel 9_28_2018.xlsx', sheet_name="HUM")
wind_in = pd.read_excel(location+'Weather Data for Drexel 9_28_2018.xlsx', sheet_name="WSP")
cloud_in = pd.read_excel(location+'Weather Data for Drexel 9_28_2018.xlsx', sheet_name="CC")

# other data
customer_in = pd.read_excel(location+'PECO Zip Customer 2018.10.01 v2.xlsx', sheet_name="Account")
ratecode_in = pd.read_excel(location+'Rate Codes for Drexel 9_28_2018.xlsx')


In [54]:
# merge use data
daily_in = [use_oct_in, use_nov_in, use_dec_in, use_feb_in, use_mar_in, 
            use_apr_in, use_may_in, use_jun_in, use_aug_in, use_sep_in]
daily = pd.concat(daily_in)
daily = daily.rename(columns={'DAccountID':'DACCOUNTID', 'DMeterNo':'DMETERNO',
                              'DAILY_INTERVAL_USAGE':'Use'})

hourly_in = [use_jan_in, use_jul_in]
hourly = pd.concat(hourly_in)

In [55]:
# find gas records
daily = daily.loc[daily['UOM'] == 'CCF']
daily = daily.drop(columns=['UOM'])
hourly = hourly.loc[hourly['UOM'] == 'CCF']
hourly = hourly.drop(columns=['UOM'])

customer = customer_in.loc[customer_in['FUELTYPE'] == 'GAS']
customer = customer.drop(columns=['CITY', 'STATE', 'ZIPCODE', 'FUELTYPE', 'COUNTYCODE'])

# convert to datetime
daily['Dt'] =  pd.to_datetime(daily['METERREADDATE'])
hourly['Dt'] =  pd.to_datetime(hourly['METERREADDATE'])

## Data Restructuring ##

In [56]:
# functions for naming consistency
def decrement(x, startswith, split):
    """
    decrements a passed string of form "demo#" by 1
    
    Parameters
    ----------
    x : string to be decremented
    split : string to split on

    Returns
    ----------
    y : decremented string
    """
    if x.startswith(startswith):
        a,b = x.split(split)
        b = int(b)-1
        y = a + split + str(b)

        return y

    else:
        return x

    
def interval_to_hour(df):
    """
    function for fast rename/relabel during tidying process
    
    Parameters
    ----------
    df : pandas data frame

    Returns
    ----------
    df : data frame with updated column names
    """
    
    df = df.rename(columns=lambda x: decrement(x, "INTERVAL_", "_"))
    df = df.rename(columns=lambda x: x.replace("INTERVAL_", "HR"))
    return df

In [57]:
# rename for consistency
hourly = interval_to_hour(hourly)
hourly = hourly.drop(columns=['METERREADDATE','HR24'])
daily = daily.drop(columns=['METERREADDATE'])

In [58]:
# Tidy / Stack data (transform into tall data - one row per customer per hour):
# ref: http://www.jeannicholashould.com/tidy-data-in-python.html
tidy_hourly = pd.melt(hourly, 
                      id_vars=['DACCOUNTID','DMETERNO','Dt'],
                      var_name='Hour', value_name='Use')

tidy_hourly_agg = tidy_hourly.groupby(['DACCOUNTID','DMETERNO','Dt']).sum().reset_index()
# tidy_hourly_agg.head()

In [59]:
# join hourly & daily
dfs = [tidy_hourly_agg, daily]
use = pd.concat(dfs, keys=['DACCOUNTID', 'DMETERNO', 'Dt', 'Use'], sort=True).reset_index()

In [60]:
# sort by date & time
use = use.sort_values(by=["Dt"])

In [61]:
# clean up extra indexes from merging
use = use.drop(columns=['level_0','level_1'])

In [66]:
bkup=use
#use=bkup
use.head()

,DACCOUNTID,DMETERNO,Dt,Use
524144,80949142571848,610057844548,2017-10-01,0.00
578510,211377922317648,606738681120,2017-10-01,2.00
578520,98934349902496,4465121455628,2017-10-01,0.00
578522,112327789077744,4464581930016,2017-10-01,1.14
511859,296892425243660,759308730744,2017-10-01,2.28


In [67]:
# drop records with missing data
if len(use) == len(use.dropna(subset=['Use'])):
    print(f'There is no missing data in the {len(use)} rows')
else:
    use = use.dropna(subset=['Use'])

There is no missing data in the 2402840 rows


In [90]:
ids = use.loc[:, use.columns.isin(['DACCOUNTID','DMETERNO'])].drop_duplicates()
print(f"There are {len(use['DACCOUNTID'].drop_duplicates())} unique AccountIDs in the data")
print(f"There are {len(use['DMETERNO'].drop_duplicates())} unique MeterNos in the data")
print(f'There are {len(ids)} unique AccountID / MeterNo pairs in the data')

There are 8187 unique AccountIDs in the data
There are 7047 unique MeterNos in the data
There are 8369 unique AccountID / MeterNo pairs in the data


In [99]:
# to find records with full year, map each (AccountID,MeterNo) pair to a year of most-recent dates
from datetime import datetime, timedelta
recent = max(use['Dt'])
recent_year = pd.date_range(start=recent-timedelta(days=365), end=recent)

In [117]:
# check each account pair for full recent year
daily_use = pd.DataFrame(columns=[list(use)])
for id in range(1,ids.size):
    # subset use by id
    use_id = use.loc[(use['DACCOUNTID'] == ids.values[i,0]) & (use['DMETERNO'] == ids.values[i,1])]
    
    # check use_id['Dt'] contains recent_year
    # if no, drop data?
    # if yes, check for duplicates
        # deal with duplicates
        # append to new df?
        #daily_use = pd.concat([daily_use, use_id], axis=1)
    

              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-

              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-

              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-

              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-

              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-

...

              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-

              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-

              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-

              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-30  0.0
              DACCOUNTID       DMETERNO         Dt  Use
2393994  160412444493832  5162316082564 2018-09-

KeyboardInterrupt: 

In [68]:
# identify unique ids/meterno pairs
def uniqueIDs(dfs, cols):
    """
    Identifies uniques in specified cols and intersects across dfs to return only the overlap
    
    Parameters
    ----------
    dfs : list of pandas data frames with consistent headers
    cols : list of column names to track

    Returns
    ----------
    ids : pandas df with cols columns containing only ids present in all dfs
    """
    
    dfs2 = []
    for df in dfs:
        dfs2.append(df.loc[:, df.columns.isin(cols)].drop_duplicates())
   
    ids = dfs2[0]
    for i in range(1,len(dfs2)):
        ids = pd.merge(ids, dfs2[i], how='inner')
        
    return ids

In [70]:
# identify unique ids/meterno pairs
daily_ids = uniqueIDs(use,['DACCOUNTID','DMETERNO'])
print(len(daily_ids))
daily_ids.head()
#daily_ids = daily_ids.rename(columns={'DAccountID':'DACCOUNTID', 'DMeterNo':'DMETERNO'})

#hourly_ids = uniqueIDs(hourly_in,['DACCOUNTID','DMETERNO'])

#ids = uniqueIDs([daily_ids,hourly_ids],['DACCOUNTID','DMETERNO'])

AttributeError: 'str' object has no attribute 'loc'

In [ ]:
# cross-reference ids with sufficient data against rate code list
# rate_codes = ids.merge(customer, how='inner', on=['DACCOUNTID','DMETERNO'])

# we need to use customer_in instead of customer because customer filters out electric rate codes, some of whom still use gas
rate_codes = ids.merge(customer_in, how='inner', on=['DACCOUNTID','DMETERNO'])

print(len(ids))
print(len(rate_codes))
print(f"There is {100*len(ids)/len(rate_codes)} percent overlap between customer rate code info and gas customer use info")

In [ ]:
# Add dummy variables for day-of-week
use = use.join(pd.get_dummies(use['Dt'].dt.weekday_name))

In [ ]:
# Add dummy variables for holidays
from pandas.tseries.holiday import USFederalHolidayCalendar
cal = USFederalHolidayCalendar()
dr = pd.date_range(start=min(use['Dt']), end=max(use['Dt']))
holidays = cal.holidays(start=dr.min(), end=dr.max())

use = use.join(pd.get_dummies(use['Dt'].isin(holidays)))
use = use.rename(columns={True:'Holiday'})
use = use.drop(columns=[False])

In [ ]:
# save file(s) as pickles
# using save location to save with other data files outside of git repo
# data.to_csv(location+'peco.csv', sep='\t')
daily.to_pickle(location+'peco_daily.pkl.zip')
ids.to_pickle(location+'peco_daily_ids.pkl.zip')
rate_codes.to_pickle(location+'peco_daily_rate_codes.pkl.zip')

In [ ]:
# Merge
weather1 = pd.merge(temp, humid, how='inner', on=['Dt'])
weather2 = pd.merge(wind, cloud, how='inner', on=['Dt'])
weather = pd.merge(weather1, weather2, how='inner', on=['Dt'])

daily = pd.merge(use, weather, how='inner', on=['Dt'])
# CustIDs || Date | Consumption |||| Weather_variables 